# Conexión Mongo-Redis-PANDAS

## Instalar librarías:

    conda activate bigdata
    conda install -c conda-forge pymongo redis-py

### Importar librerías

In [46]:
import redis
from pymongo import MongoClient
import pandas as pd

In [52]:
def conexion_mongo():
    HOST="XXX"
    PORT=27017
    USERNAME="XXX"
    PASSWORD="XXX"
    cli_mongo = MongoClient(f"mongodb+srv://{USERNAME}:{PASSWORD}@{HOST}/test")
    return cli_mongo

def conexion_redis():
    redis_cli = redis.Redis()
    #redis_cli.auth("123quetal123")
    #redis_cli.ping()
    return redis_cli

#Conexión Mongo
cli_mongo=conexion_mongo()
# @ sample_guides -> planets
db_guides=cli_mongo['sample_guides']
coleccion_planetas=db_guides['planets']

#Conexión Redis
redis_cli = conexion_redis()
redis_cli.flushall()

aux_documentos = coleccion_planetas.find({})
for documento in aux_documentos:
    idPlaneta = documento.pop('_id')
    documento.update({"_id": str(idPlaneta) })
    redis_cli.json().set(documento.get('name'), "$", documento)

listaPlanetas = []

for i in redis_cli.scan_iter():
    auxPlaneta = redis_cli.json().get(i.decode(), '$')
    listaPlanetas.append(auxPlaneta[0])

df = pd.DataFrame(listaPlanetas)

df

,name,orderFromSun,hasRings,mainAtmosphere,surfaceTemperatureC,_id
0,Neptune,8,True,"[H2, He, CH4]","{'min': None, 'max': None, 'mean': -201}",621ff30d2a3e781873fcb65f
1,Mars,4,False,"[CO2, Ar, N]","{'min': -143, 'max': 35, 'mean': -63}",621ff30d2a3e781873fcb65e
2,Earth,3,False,"[N, O2, Ar]","{'min': -89.2, 'max': 56.7, 'mean': 14}",621ff30d2a3e781873fcb661
3,Saturn,6,True,"[H2, He, CH4]","{'min': None, 'max': None, 'mean': -139.15}",621ff30d2a3e781873fcb663
4,Uranus,7,True,"[H2, He, CH4]","{'min': None, 'max': None, 'mean': -197.2}",621ff30d2a3e781873fcb65d
5,Venus,2,False,"[CO2, N]","{'min': None, 'max': None, 'mean': 464}",621ff30d2a3e781873fcb662
6,Mercury,1,False,[],"{'min': -173, 'max': 427, 'mean': 67}",621ff30d2a3e781873fcb65c
7,Jupiter,5,True,"[H2, He, CH4]","{'min': None, 'max': None, 'mean': -145.15}",621ff30d2a3e781873fcb660


## De Mongo DB a PANDAS

In [ ]:
dfPlanetasDirectos = pd.DataFrame.from_records(coleccion_planetas.find())
dfPlanetasDirectos